In [ ]:
import time as ti
import pandas as pd
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

In [ ]:
driver_path = 'msedgedriver.exe'
service = Service(driver_path)
driver = webdriver.Edge(service=service)

In [ ]:
URL_BASE = "https://www3.tjrj.jus.br/ejuris/ConsultarJurisprudencia.aspx"

# TODO 
# Transformar isso em um input do MCP (reestruturar como tool)
target = "danos morais"

In [ ]:
search_box_name = "ctl00$ContentPlaceHolder1$txtTextoPesq"
tables_xpath = "/html/body/div[1]/div/div[2]/form/div[3]/span/table/tbody/tr[6]/td"

In [ ]:
driver.get(URL_BASE)
ti.sleep(2)


search_box = driver.find_element(By.NAME, search_box_name)
search_box.send_keys(target)
search_box.send_keys(Keys.ENTER)

ti.sleep(5)

# Encontrar todas as tabelas
# tables = soup.find_all("table")

elem = driver.find_element(By.XPATH, tables_xpath)
html_content = elem.get_attribute("innerHTML")
soup = BeautifulSoup(html_content, "html.parser")
tables = soup.find_all("table")

data = []

for table in tables:
    lines = table.find_all("tr")

    # Ignora tabelas menores da estrutura do site (ruídos)
    if len(lines) >= 14:
        processo_info = lines[3].get_text(strip=True)
        processo_num, processo_tipo = processo_info.split('- ')
        ementa = lines[5].get_text(strip=True)
        julgamento_info = lines[6].get_text(strip=True)
        desembargador, julg_data, camara = julgamento_info.split(' - ')
        desembargador = desembargador.replace('Des(a). ', '').strip()
        julg_data = julg_data.replace('Julgamento: ', '').strip()
        resumo = lines[10].get_text(" ", strip=True)  # usa separador para manter espaços
        publicacao = lines[13].get_text(strip=True)
        publicacao = publicacao.split("Data de Publicação: ")[-1].removesuffix("(*)").strip()

        # Adiciona no dataset
        data.append({
            "Número do Processo": processo_num,
            "Tipo do Processo": processo_tipo,
            "Ementa": ementa,
            "Desembargador": desembargador,
            "Data do Julgamento": julg_data,
            "Câmara": camara,
            "Resumo": resumo,
            "Data da Publicação": publicacao
        })

df = pd.DataFrame(data)

display(df)

driver.quit()
